In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

from sklearn.preprocessing import StandardScaler,MinMaxScaler 


In [3]:
#tail -n +2 train.csv | split -l 150000


In [4]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [9]:
splits = splits[0:256]

In [5]:
splits

['xzauf',
 'xzdkn',
 'xzaak',
 'xzbgq',
 'xzcuo',
 'xzcwy',
 'xzcky',
 'xzehm',
 'xzaqi',
 'xzayj',
 'xzdlo',
 'xzehs',
 'xtw',
 'xzdew',
 'xzaqr',
 'xzbbo',
 'xet',
 'xzecc',
 'xzcuw',
 'xtp',
 'xzdfm',
 'xzcsn',
 'xzfbf',
 'xzbvv',
 'xzbmr',
 'xzeeb',
 'xzdrk',
 'xdy',
 'xzbci',
 'xra',
 'xxj',
 'xzffy',
 'xzagu',
 'xzamy',
 'xzczg',
 'xzdwm',
 'xjv',
 'xlu',
 'xft',
 'xzdif',
 'xzdpw',
 'xzbun',
 'xzaqh',
 'xzfap',
 'xzadw',
 'xzcqg',
 'xzatz',
 'xzelh',
 'xzbwp',
 'xzeot',
 'xzbrf',
 'xzees',
 'xzddf',
 'xzegg',
 'xzfet',
 'xzbjb',
 'xzcvz',
 'xzefu',
 'xzbhz',
 'xdd',
 'xzbft',
 'xzddz',
 'xzanw',
 'xzaxn',
 'xip',
 'xzaku',
 'xzbhd',
 'xzctx',
 'xzcgk',
 'xgf',
 'xun',
 'xzdzb',
 'xxt',
 'xzdkh',
 'xzfgi',
 'xzdyx',
 'xzffi',
 'xrh',
 'xzdlz',
 'xzach',
 'xzbcc',
 'xzbkw',
 'xir',
 'xzcug',
 'xzbfh',
 'xzbqp',
 'xzdzc',
 'xzcbm',
 'xzffr',
 'xzcol',
 'xzddu',
 'xzeym',
 'xzapb',
 'xzafg',
 'xzfcu',
 'xzfdy',
 'xzakw',
 'xkq',
 'xzbpq',
 'xzbyp',
 'xzasu',
 'xzevj',
 'xzchc',
 'xz

In [9]:
TIMESTEPS = 50


rows = np.array_split(list(range(0,150000)),int(150000/TIMESTEPS))

In [10]:
rows[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [43]:
a = [[1,2,3],[4,5,6],[6,7,8]]

np.delete(a, 1, axis=0).reshape(-1)

array([1, 2, 3, 6, 7, 8])

In [12]:
len(splits)

4195

In [9]:
for i,split in enumerate(splits):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [13]:
len(splits)

256

In [11]:
BATCH_SIZE=len(splits)

def get_segment_data(split, sequence_idx):
    columns = ['acoustic_data','time_to_failure']
    features = ['acoustic_data','rolling_100']

    path = 'train/%s' % (split)
    #print("DEBUGGGGG")
    #print(path)
    skipped_rows_indices = get_skipped_rows(sequence_idx)
    df = pd.read_csv(path, float_precision='round_trip', header=None, skiprows=skipped_rows_indices)
    df.columns = columns
    df['rolling_100'] = df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    df['acoustic_data'] = df['acoustic_data'].astype(float)
    df[features] = StandardScaler().fit_transform(df[features])
    return df[features].values, df['time_to_failure'].values[-1]


def get_skipped_rows(sequence_idx):
    return np.delete(rows, sequence_idx, axis=0).reshape(-1)



def get_generated_batch(sequence_idx):
    batch = np.empty((len(splits),TIMESTEPS,2),dtype=float)
    targets = np.empty((len(splits),1),dtype=float)
    for i,split in enumerate(splits):
        segment_data, target = get_segment_data(split, sequence_idx)
        batch[i] = segment_data
        targets[i] = target
    return batch, targets

In [18]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
split_size = 500


def get_sequence(sequence_idx, sequence_batch_size):
    print("DEBUGGGGG 1")
    splits_chunk = splits[sequence_idx:sequence_idx+sequence_batch_size]
    print("DEBUGGGGG 2")

    training, ys = read_raw_training_data(splits_chunk)
    print("DEBUGGGGG 3")
    new_training, new_ys = fornat_training_data(training, ys)
    print("DEBUGGGGG 4")
    train, Y = swap_axes(new_training, new_ys)
    print("DEBUGGGGG 5")

    return train, Y
    
    
def read_raw_training_data(splits_chunk): 
    training = []
    ys = []

    features = ['acoustic_data','time_to_failure']

    for chunk in splits_chunk:
        path = 'train/%s' % (chunk)
        #print("DEBUGGGGG")
        #print(path)
        chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
        chunk_df.columns = features
        chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
        chunk_df[features] = StandardScaler().fit_transform(chunk_df[features])
        training.append(chunk_df[features].values)
        ys.append(chunk_df['time_to_failure'].values)

    training = np.array(training)
    ys = np.array(ys)
    
    return training, ys

def fornat_training_data(training, ys):
    new_training = []
    new_ys = []
    for i,sequence in enumerate(training):
        #split each sequence in split_size
        splits = np.array_split(sequence, split_size)
        #print(np.array(splits).shape)
        new_training.append(splits)
        splits_y = np.array_split(ys[i], split_size)
        new_ys.append(splits_y)


    new_training = np.array(new_training)
    new_ys = np.array(new_ys)
    return new_training, new_ys


def swap_axes(new_training,new_ys):
    train = np.swapaxes(new_training,0,1)
    train = train.reshape(-1,300,2)
    
    targets = np.swapaxes(new_ys,0,1)
    
    Y = []
    for target in targets:
        Y.append(np.array(target[:,-1]))

    Y = np.array(Y).reshape(-1)
    
    return train, Y

In [8]:
splits_chunk = splits[0:200]

In [19]:
len(splits)

4195

In [16]:
training = np.empty((200,150000,2),dtype=float)
ys = np.empty((200,150000),dtype=float)

features = ['acoustic_data','time_to_failure']


for i,chunk in enumerate(splits_chunk):
    path = 'train/%s' % (chunk)
    chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
    chunk_df.columns = features
    chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    chunk_df[features] = StandardScaler().fit_transform(chunk_df[features])
    training[i] = chunk_df[features].values
    ys[i] = chunk_df['time_to_failure'].values

training = np.array(training)
ys = np.array(ys)

/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataCon

/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataCon

/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataCon

/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/stephane/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataCon

ValueError: could not broadcast input array from shape (45480,2) into shape (150000,2)

In [17]:
ys.shape

(200, 150000)

In [14]:
training.shape

(200, 150000, 2)

In [8]:
#number of batches must cover a sequence length i.e. 100 * 1500 = 15000, batch size = 1500 

In [18]:
split_size = 500
new_training = []
new_ys = []
for i,sequence in enumerate(training):
    #split each sequence in split_size
    splits = np.array_split(sequence, split_size)
    #print(np.array(splits).shape)
    new_training.append(splits)
    splits_y = np.array_split(ys[i], split_size)
    new_ys.append(splits_y)

    
new_training = np.array(new_training)
new_ys = np.array(new_ys)

In [19]:
new_ys.shape

(200, 500, 300)

In [ ]:
a = np.array([[[1,2,3], [4,5,6]], [[10,20,30], [40,50,60]]])
print(a.shape)
np.array(a[0])[:,-1]

In [ ]:
a = np.array([[1,2,3], [4,5,6]])
a
a.reshape(-1)

In [153]:
np.array(new_ys[0])[:,-1].shape

IndexError: too many indices for array

In [28]:
new_training.shape

(200, 500, 300, 2)

In [20]:
train = np.swapaxes(new_training,0,1)

In [21]:
train = train.reshape(-1,300,2)

In [22]:
targets = np.swapaxes(new_ys,0,1)

In [23]:
Y = []
for target in targets:
    Y.append(np.array(target[:,-1]))
    
Y = np.array(Y).reshape(-1)

In [13]:
Y.shape

(100000,)

In [27]:
train.shape

(100000, 300, 2)

In [30]:
a = np.array([[[[1],[2],[3],[4]],[[5],[6],[7],[8]],[[8],[9],[10],[11]]], [[[10],[20],[30],[40]],[[50],[60],[70],[80]],[[80],[90],[100],[110]]],\
              [[[1000],[2000],[3000],[4000]],[[5000],[6000],[7000],[8000]],[[8000],[9000],[10000],[11000]]] ])
print(a.shape)
print(a)

np.swapaxes(a,0,1)

(3, 3, 4, 1)
[[[[    1]
   [    2]
   [    3]
   [    4]]

  [[    5]
   [    6]
   [    7]
   [    8]]

  [[    8]
   [    9]
   [   10]
   [   11]]]


 [[[   10]
   [   20]
   [   30]
   [   40]]

  [[   50]
   [   60]
   [   70]
   [   80]]

  [[   80]
   [   90]
   [  100]
   [  110]]]


 [[[ 1000]
   [ 2000]
   [ 3000]
   [ 4000]]

  [[ 5000]
   [ 6000]
   [ 7000]
   [ 8000]]

  [[ 8000]
   [ 9000]
   [10000]
   [11000]]]]


array([[[[    1],
         [    2],
         [    3],
         [    4]],

        [[   10],
         [   20],
         [   30],
         [   40]],

        [[ 1000],
         [ 2000],
         [ 3000],
         [ 4000]]],


       [[[    5],
         [    6],
         [    7],
         [    8]],

        [[   50],
         [   60],
         [   70],
         [   80]],

        [[ 5000],
         [ 6000],
         [ 7000],
         [ 8000]]],


       [[[    8],
         [    9],
         [   10],
         [   11]],

        [[   80],
         [   90],
         [  100],
         [  110]],

        [[ 8000],
         [ 9000],
         [10000],
         [11000]]]])

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(train, Y, test_size=0.1, random_state=42)

In [13]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self, split_filenames):
        self.split_filenames = split_filenames

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return int(150000 / TIMESTEPS)

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
        train,Y = get_generated_batch(idx)
        print("idx",idx)
        print("LOLILOL")
        print(train.shape, Y.shape)

        return (train,Y)

In [14]:
train.shape

(100000, 300, 2)

In [15]:
Y.shape

(100000,)

In [16]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


dropout=0.2


my_model = Sequential()

my_model.add(CuDNNLSTM(#use_bias = True,unit_forget_bias=True,\
                  units = 3,\
                  stateful=True,batch_input_shape=(BATCH_SIZE,TIMESTEPS,2),\
                  #dropout=dropout,recurrent_dropout=dropout,
                  ))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0)
]

#history = my_model.fit(train, Y, batch_size=BATCH_SIZE, epochs=100, shuffle=False,
                      #validation_data=(val_data,y_val), callbacks=callbacks
                    #  )
my_training_batch_generator = MY_Generator(splits)
        
my_model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=500,
                                          epochs=10,
                                          shuffle=False,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          use_multiprocessing=True,
                                          workers=8,
                                          #max_queue_size=32
                      )

import matplotlib.pyplot as plt


#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_4 (CuDNNLSTM)     (4195, 3)                 84        
_________________________________________________________________
dense_4 (Dense)              (4195, 1)                 4         
Total params: 88
Trainable params: 88
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
idx 0
LOLILOL
(4195, 50, 2) (4195, 1)
idx 0
LOLILOL
(4195, 50, 2) (4195, 1)
idx 6
LOLILOL
(4195, 50, 2) (4195, 1)
idx 1
LOLILOL
(4195, 50, 2) (4195, 1)
  1/500 [..............................] - ETA: 25:58:27 - loss: 5.7942 - mean_absolute_error: 5.7942idx 4
LOLILOL
(4195, 50, 2) (4195, 1)
idx 3
LOLILOL
(4195, 50, 2) (4195, 1)
idx 2
LOLILOL
(4195, 50, 2) (4195, 1)
  3/500 [..............................] - ETA: 9:00:35 - loss: 5.7915 - mean_absolute_error: 5.7915 idx 5
LOLILOL
(4195, 50, 2) (4195, 1)
  6/500 [................

Process ForkPoolWorker-10:
Process ForkPoolWorker-5:
Process ForkPoolWorker-11:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Proce

  File "/home/stephane/.local/lib/python3.6/site-packages/pandas/core/series.py", line 169, in __init__
    index = ensure_index(index)
Process ForkPoolWorker-12:
  File "/home/stephane/.local/lib/python3.6/site-packages/pandas/io/parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/stephane/.local/lib/python3.6/site-packages/pandas/core/indexing.py", line 1154, in _get_listlike_indexer
    indexer = ax.get_indexer_for(key)
  File "<ipython-input-13-365a8eeac62b>", line 17, in __getitem__
    train,Y = get_generated_batch(idx)
  File "/home/stephane/.local/lib/python3.6/site-packages/pandas/io/parsers.py", line 1122, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/home/stephane/.local/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 4454, in get_indexer_for
    return self.get_indexer(target

KeyboardInterrupt: 

In [13]:
next(my_training_batch_generator)

TypeError: 'MY_Generator' object is not an iterator